In [ ]:
import os
import numpy as np
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
import re
import pandas as pd

In [ ]:
def link_extractor(link):
    os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    os.environ['PATH']

    options = Options()
    options.set_headless(headless=True)
    driver = webdriver.Firefox(firefox_options=options)
    
    driver.get(link)
    
    # Get total pages
    total_page=int(driver.find_element_by_id('resultPageNavigation').text.split()[-1])
    
    # Get Url
    url=driver.find_elements_by_class_name('page')[1].find_element_by_class_name('rs-btn').get_attribute('href')

    # Generate url list
    url_list=[url.replace('?pg=2',f'?pg={i}') for i in range(1,total_page+1)]
    
    driver.close()

    return url_list

In [ ]:
pages=link_extractor('https://www.thomann.de/es/otros_tipos_de_construccion.html')

In [ ]:
def guitar_info(page):
    os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    os.environ['PATH']

    options = Options()
    options.set_headless(headless=True)
    driver = webdriver.Firefox(firefox_options=options)    

    driver.get(page)
    
    # Extract guitar info
    ginfo=[]
    for i in driver.find_elements_by_xpath('//*[@id="defaultResultPage"]/div'):
        g_info=re.split('\n',i.text)
        # Check if not have reviews
        if g_info[1].isnumeric() != True:
            g_info.insert(1,0)
        # Extract Url guitar
        g_info.append(i.find_element_by_class_name('article-link.link').get_attribute('href'))
        # Extract image
        g_info.append(i.find_element_by_class_name('image').get_attribute('src'))
        ginfo.append(g_info)  
        
    print(len(ginfo))    
    time.sleep(1)
    driver.close()
    
    return ginfo    

In [ ]:
def guitar_info(pages):
    ginfo=[]
    for count, page in enumerate(pages):
        
        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
        os.environ['PATH']

        options = Options()
        options.set_headless(headless=True)
        driver = webdriver.Firefox(firefox_options=options)    
        
        driver.get(page)

        print(f'Page {count+1} - Starts: {time.ctime()}')


        # Extract guitar info
        
        for i in driver.find_elements_by_xpath('//*[@id="defaultResultPage"]/div'):
            g_info=re.split('\n',i.text)
            # Check if not have reviews
            if g_info[1].isnumeric() != True:
                g_info.insert(1,0)
            # Extract Url guitar
            g_info.append(i.find_element_by_class_name('article-link.link').get_attribute('href'))
            # Extract image
            g_info.append(i.find_element_by_class_name('image').get_attribute('src'))
            ginfo.append(g_info)  

        print(len(ginfo))    
        time.sleep(1)
        driver.close()
        
        print(f'Ends: {time.ctime()}')
    
    return ginfo 

In [ ]:
ginfo=guitar_info(pages)

In [ ]:
# Clean cols
def clean_cols(list_guitar_info):
    # Check length
    check=[len(i) for count, i in enumerate(list_guitar_info)]
    
    for num in pd.unique(check):
        if num != 9 and num != 11 and num != 12:
            break
        else:
            pass
    
        for count, i in enumerate(list_guitar_info):
            if len(i)==11:
                del i[2:5]
                del i[3:6]
            elif len(i)==12:
                del i[2:6]
                del i[3:6]
            elif len(i)==9:
                del i[2]
                del i[3:6] 

In [ ]:
def rating_guitar(g_info_list):
    rating=[]
    for count, url_guitar in enumerate(g_info_list):
        os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'

        os.environ['PATH']


        options = Options()
        options.set_headless(headless=True)
        driver = webdriver.Firefox(firefox_options=options)

        print(f'Element {count+1} - Starts: {time.ctime()}')
        driver.get(url_guitar[3])
        try:
            rating.append(driver.find_element_by_class_name('ratingValue').text)
        except:
            rating.append(0)

        driver.close()
        print(f'Ends: {time.ctime()}')
        
    return rating

In [ ]:
clean_cols(ginfo)

In [ ]:
rating=rating_guitar(ginfo)

In [ ]:
# Make DataFrame
def data_model(ginfo,rating):
    data=pd.DataFrame(ginfo,columns=['Guitar','Reviews','Price','Url_Product','Url_image'])
    data['Rating']=rating
    return data

In [ ]:
strato_data=data_model(ginfo,rating)

In [ ]:
# For all products with 2 prices (Discount / Old) get the actual Price
def clean_price(df):
    return df.apply(lambda x : re.split('(?<=\€)(.*?)(?=\\w)' ,x)[0] 
                    if len(re.split('(?<=\€)(.*?)(?=\\w)' ,x)[0])>1 else x)

In [ ]:
# Extract all brands in list
def brand_extractor(url_class):
    os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
    os.environ['PATH']

    options = Options()
    options.set_headless(headless=True)
    driver = webdriver.Firefox(firefox_options=options)    

    driver.get(url_class)
    
    brand_list = [i.get_attribute('value') for i in driver.find_elements_by_xpath('//*[@id="keyfeaturefilter-manufacturer-select"]/option')]
    
    driver.close()
    
    return brand_list

In [ ]:
brand_list=brand_extractor('https://www.thomann.de/es/modelos_st.html?viewMode=block')

In [ ]:
strato_data['Price']=clean_price(strato_data['Price'])

In [ ]:
# Get Brand & Model from guitar
def get_df_brand_model(data, brand_list):
    l=[re.split(fr'({brand})',i)[1:]  for i in data for brand in brand_list if re.findall(brand,i)==[brand]]
    return pd.DataFrame(l, columns=['Brand','Model'])

In [ ]:
df=get_df_brand_model(strato_data['Guitar'],brand_list)

In [ ]:
# Save your data
strato_data.to_csv('./data/raw/thomann/stratocaster.csv',index=False)
strato_data